In [3]:
import tweepy
import json


In [9]:
with open("C:/Users/William Jiang/Documents/credentials.json") as f:
    d = json.load(f)
    cred_json=d["twitter_api"]
    

# Authenticate to Twitter
auth = tweepy.OAuthHandler(cred_json["CONSUMER_KEY"], cred_json["CONSUMER_SECRET"])
auth.set_access_token(cred_json["ACCESS_TOKEN"], cred_json["ACCESS_TOKEN_SECRET"])

# Create API object
api = tweepy.API(auth)

# Create a tweet
api.update_status("Hello Tweepy2")

Status(_api=<tweepy.api.API object at 0x000001C082E782B0>, _json={'created_at': 'Sun Jun 28 12:02:15 +0000 2020', 'id': 1277210700935380995, 'id_str': '1277210700935380995', 'text': 'Hello Tweepy2', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://en.wikipedia.org/wiki/Uniform_Resource_Locator" rel="nofollow">bumbumapp</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 4073594774, 'id_str': '4073594774', 'name': 'Will', 'screen_name': 'iamwilliamj1', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 0, 'listed_count': 0, 'created_at': 'Fri Oct 30 23:24:16 +0000 2015', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_coun